In [ ]:
!pip install ffmpeg moviepy

# **SRGAN Model Implementation :-**

---



---



In [ ]:
import timeit
import cv2 
import os
import sys
import numpy as np
from model import resolve_single
from utils import load_image, plot_sample
from model.srgan import generator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
from keras.preprocessing.image import save_img
import moviepy.editor as mp

# Input file
cam = cv2.VideoCapture("Play.mp4") 
fps = cam.get(cv2.CAP_PROP_FPS)
print("FPS = ",fps)

# To create a folder to store video frames
try:
    if not os.path.exists('data'): 
        os.makedirs('data') 
except OSError:
    print ('Error: Creating directory of data') 

currentframe = 0

# Initialize empty list to store frames
frames_list = []

print("Creating frames....")

# Convert video into frames
while(True): 
    ret,frame = cam.read() 
    if ret: 
        name = './data/frame' + str(currentframe).zfill(3) + '.jpg'
        cv2.imwrite(name, frame) 
        currentframe += 1
        frames_list.append(name)
    else: 
        break

print(f"{currentframe} Frames Created.")
cam.release() 
cv2.destroyAllWindows()

In [ ]:
start = timeit.default_timer()
model = generator()
model.load_weights('weights/srgan/gan_generator.h5')
n= len(frame_list)

# To create a folder to store super resolved frames
try:
    if not os.path.exists('out_images'): 
        os.makedirs('out_images') 
except OSError:
    print ('Error: Creating directory of data') 
    

# Initialize empty list to store super resolved frames
SRframe_list= []

# Applying SRGAN Model and storing super resolved imgaes.
for i in range(n):
  lr = load_image(frame_list[i])
  sr = resolve_single(model, lr)
  # The super resolved frames are in numpy array format, so here we will change them in an image format.
  out_name = './out_images/frame' + str(i).zfill(3) + '.jpg'
  img1 = save_img(out_name, sr)
  SRframe_list.append(out_name)
  print(f"{i} frame created...")

stop = timeit.default_timer()
print(f"Completed in : {(stop-start)/60} minutes")


In [ ]:
# Conversion of Super Resolved frames into a video
start = timeit.default_timer()
for i in range(len(SRframe_list)):
    filename=SRframe_list[i]
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)

fps = 29 
out = cv2.VideoWriter('output.mp4',cv2.VideoWriter_fourcc(*'DIVX'), fps , size)

# Writing Frames into video
for i in range(len(SRframe_list)):
    out.write(cv2.imread(SRframe_list[i]))
out.release()

stop = timeit.default_timer()
print(f"Completed in : {(stop-start)/60} minutes")

In [ ]:
# Extracting audio file from video file then combining with output video file

my_clip = mp.VideoFileClip(r"Play.mp4")  # input video file name
my_clip.audio.write_audiofile(r"audio.mp3")  # extracted audio file name
audio_background = mp.AudioFileClip("audio.mp3")  
my_clip2 = mp.VideoFileClip(r"output.mp4")  # output video file name
final_clip = my_clip2.set_audio(audio_background)
final_clip.write_videofile("output_audio.mp4",fps=fps)  # output video file with audio

In [ ]:
# Removing all stored frames
import shutil
shutil.rmtree('data')
shutil.rmtree('out_images')
os.remove("audio.mp3")

